In [28]:
import requests

KAKAO_API_KEY = "70200e2a88cc0ab7ee1632c35d8aac0b"  # 🔑 발급받은 키

headers = {
    "Authorization": f"KakaoAK {KAKAO_API_KEY}"
}

params = {
    "query": "대전 칼국수 맛집",
    "size": 5
}

url = "https://dapi.kakao.com/v2/local/search/keyword.json"
res = requests.get(url, headers=headers, params=params)
data = res.json()



In [29]:
data

{'documents': [{'address_name': '대전 동구 삼성동 304-36',
   'category_group_code': 'FD6',
   'category_group_name': '음식점',
   'category_name': '음식점 > 한식 > 국수 > 칼국수',
   'distance': '',
   'id': '16302660',
   'phone': '042-627-9972',
   'place_name': '오씨칼국수',
   'place_url': 'http://place.map.kakao.com/16302660',
   'road_address_name': '대전 동구 옛신탄진로 13',
   'x': '127.42510563102718',
   'y': '36.34203952671037'},
  {'address_name': '대전 서구 둔산동 1440',
   'category_group_code': 'FD6',
   'category_group_name': '음식점',
   'category_name': '음식점 > 한식 > 국수 > 칼국수',
   'distance': '',
   'id': '7825433',
   'phone': '042-471-0316',
   'place_name': '대선칼국수',
   'place_url': 'http://place.map.kakao.com/7825433',
   'road_address_name': '대전 서구 둔산중로40번길 28',
   'x': '127.388276620787',
   'y': '36.3499690372587'},
  {'address_name': '대전 중구 선화동 20',
   'category_group_code': 'FD6',
   'category_group_name': '음식점',
   'category_name': '음식점 > 한식 > 국수 > 칼국수',
   'distance': '',
   'id': '23346501',
   'phone

In [26]:
for place in data['documents']:
    print(place['place_name'], "-", place['address_name'])


오씨칼국수 - 대전 동구 삼성동 304-36
대선칼국수 - 대전 서구 둔산동 1440
손이가어죽칼국수 - 대전 중구 선화동 20
오씨칼국수 - 대전 동구 원동 41-3
미소본가스마일칼국수 - 대전 중구 대흥동 440-1


In [33]:
import os
import openai
import requests

# ✅ API 키 설정 (환경변수에서 불러오거나 직접 입력)
openai.api_key = os.getenv("OPENAI_API_KEY")
KAKAO_API_KEY = "70200e2a88cc0ab7ee1632c35d8aac0b"  # 또는 "직접 입력"

# ✅ 1. 사용자 입력 → 지역/키워드 추출 (OpenAI)
def extract_location_and_keyword(user_input):
    messages = [
        {
            "role": "system",
            "content": "당신은 사용자의 문장에서 지역과 음식 키워드를 추출하는 도우미입니다."
        },
        {
            "role": "user",
            "content": f"""
다음 문장에서 지역과 음식점 관련 키워드를 각각 한 줄씩 다음 형식으로 출력해줘:
지역: <지역>
키워드: <키워드>

문장: "{user_input}"
"""
        }
    ]

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0,
        max_tokens=100
    )
    result = response.choices[0].message.content
    region, keyword = "서울", "맛집"  # 기본값
    for line in result.split("\n"):
        if line.startswith("지역:"):
            region = line.replace("지역:", "").strip()
        elif line.startswith("키워드:"):
            keyword = line.replace("키워드:", "").strip()
    return region, keyword

# ✅ 2. 카카오 로컬 API로 장소 검색
def search_places(region, keyword):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": f"{region} {keyword}", "size": 5}

    res = requests.get(url, headers=headers, params=params)
    if res.status_code != 200:
        return []
    
    items = res.json().get("documents", [])
    places = []
    for place in items:
        places.append({
            "name": place["place_name"],
            "address": place["road_address_name"] or place["address_name"],
            "url": place["place_url"],
            "category": place["category_name"]
        })
    return places

# ✅ 3. 추천 문장 생성 (OpenAI)
def generate_recommendation(region, keyword, places):
    place_list = "\n".join(
        [f"{i+1}. {p['name']} - {p['address']} ({p['category']})" for i, p in enumerate(places)]
    )

    prompt = f"""
사용자는 "{region}" 지역에서 "{keyword}" 관련 음식점을 찾고 있습니다.
아래는 검색된 음식점 목록입니다:

{place_list}

이 중 3곳을 추천해 주세요. 추천 이유를 한국어로 간단하게 설명해 주세요.
"""

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "당신은 맛집 추천 도우미입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=300
    )
    return response.choices[0].message.content

# ✅ 4. 전체 실행 함수
def run_food_finder(user_input):
    region, keyword = extract_location_and_keyword(user_input)
    places = search_places(region, keyword)
    if not places:
        return "❌ 음식점 검색 결과가 없습니다."
    return generate_recommendation(region, keyword, places)

# ✅ 5. 예제 실행
if __name__ == "__main__":
    query = input("📍 음식점 관련 질문을 입력하세요:\n> ")
    answer = run_food_finder(query)
    print("\n🤖 추천 결과:\n")
    print(answer)



🤖 추천 결과:

거제 장평 지역에서 추천드릴 맛집은 다음과 같습니다:

1. **장터국밥 장평점** - 이곳은 국밥으로 유명한 곳으로, 든든하고 맛있는 한식을 즐기기에 좋습니다. 진하게 우려낸 국물과 푸짐한 고기의 조화가 일품입니다.

2. **목구멍 장평점** - 육류, 특히 고기를 좋아하신다면 이곳을 추천드립니다. 신선한 고기와 다양한 한식 반찬이 함께 제공되어 맛있는 고기 요리를 즐기기에 적합합니다.

3. **첨단돌솥감자탕 거제장평점** - 감자탕을 좋아하신다면 이곳이 제격입니다. 깊고 진한 국물과 부드러운 감자가 조화를 이루어, 든든하고 따뜻한 한 끼 식사를 하기에 좋습니다.

각 음식점은 한식의 다양한 맛을 경험할 수 있는 곳으로, 원하는 취향에 맞춰 선택하시면 좋을 것 같습니다.
